In [134]:
#!pip install trueskill
from trueskill import Rating, rate, quality_1vs1, rate_1vs1, quality
from collections import defaultdict, Counter
from itertools import groupby
from pprint import pprint
import random
from numpy.random import choice

In [135]:
# modified example of one-on-one from https://trueskill.org/

# assign Alice and Bob's ratings
a = Rating(25)  
b = Rating(25)

a, b = rate_1vs1(a, b, drawn=False)
print('a:', a, 'b:', b)
a, b = rate_1vs1(b, a, drawn=False)
print('a:', a, 'b:', b)
a, b = rate_1vs1(b, a, drawn=False)
print('a:', a, 'b:', b)

# print(quality_1vs1(a, b))
# print('{:.1%} chance to draw'.format(quality_1vs1(a, b)))
# game_results = Counter()
# for i in range(100):
#     # play n games and decide on a the outcome given the probabilities
#     game_result = choice(['a_wins', 'b_wins', 'draw'], p=[0.8, 0.01, 0.19])
#     game_results[game_result] += 1
#     if game_result == 'a_wins':
#         a, b = rate_1vs1(a, b, drawn=False)  # update the ratings after the match (a beats b)
#     elif game_result == 'b_wins':
#         b, a = rate_1vs1(b, a, drawn=False)  # update the ratings after the match (b beats a)
#     else:
#         a, b = rate_1vs1(a, b, drawn=True)  # update the ratings after the match (draw)

# print(pretty_print(dict(game_results)))
# print('{:.1%} chance to draw'.format(quality_1vs1(a, b)))
# print('Player a', a)
# print('Player b', b)

a: trueskill.Rating(mu=29.396, sigma=7.171) b: trueskill.Rating(mu=20.604, sigma=7.171)
a: trueskill.Rating(mu=26.643, sigma=6.040) b: trueskill.Rating(mu=23.357, sigma=6.040)
a: trueskill.Rating(mu=27.085, sigma=5.261) b: trueskill.Rating(mu=22.915, sigma=5.261)


In [136]:
new_r1, new_r2 = rate_1vs1(b, a)
print(new_r1)
print(new_r2)

trueskill.Rating(mu=26.281, sigma=4.630)
trueskill.Rating(mu=23.719, sigma=4.630)


In [137]:
# example of free-for-all from https://stackoverflow.com/questions/24723975/python-trueskill-using-dictionaries

results = [(1, 'a', 2),
           (1, 'b', 1),
           (1, 'e', 3),
           (2, 'a', 1),
           (2, 'e', 1),
           (3, 'b', 1),
           (3, 'c', 2),
           (3, 'a', 3),
           (3, 't', 4),
           (3, 'e', 5)]
ratings = defaultdict(Rating)

for game_id, result in groupby(results, lambda x: x[0]):
#     print(game_id)
    result = list(result)
#     print(result)
    rating_groups = [(ratings[name],) for game_id, name, rank in result]
    print(rating_groups)
    ranks = [rank for game_id, name, rank in result]
    print('ranks', ranks)
    transformed_groups = rate(rating_groups, ranks=ranks)
    print(transformed_groups)
    for x, (game_id, name, rank) in enumerate(result):
        ratings[name], = transformed_groups[x]

pprint(dict(ratings))

[(trueskill.Rating(mu=25.000, sigma=8.333),), (trueskill.Rating(mu=25.000, sigma=8.333),), (trueskill.Rating(mu=25.000, sigma=8.333),)]
ranks [2, 1, 3]
[(trueskill.Rating(mu=25.000, sigma=6.208),), (trueskill.Rating(mu=31.675, sigma=6.656),), (trueskill.Rating(mu=18.325, sigma=6.656),)]
[(trueskill.Rating(mu=25.000, sigma=6.208),), (trueskill.Rating(mu=18.325, sigma=6.656),)]
ranks [1, 1]
[(trueskill.Rating(mu=22.815, sigma=5.092),), (trueskill.Rating(mu=20.836, sigma=5.257),)]
[(trueskill.Rating(mu=31.675, sigma=6.656),), (trueskill.Rating(mu=25.000, sigma=8.333),), (trueskill.Rating(mu=22.815, sigma=5.092),), (trueskill.Rating(mu=25.000, sigma=8.333),), (trueskill.Rating(mu=20.836, sigma=5.257),)]
ranks [1, 2, 3, 4, 5]
[(trueskill.Rating(mu=36.119, sigma=5.434),), (trueskill.Rating(mu=29.226, sigma=5.342),), (trueskill.Rating(mu=23.967, sigma=4.088),), (trueskill.Rating(mu=21.013, sigma=5.150),), (trueskill.Rating(mu=16.740, sigma=4.438),)]
{'a': trueskill.Rating(mu=23.967, sigma=4.0

In [138]:
from collections import defaultdict
from itertools import groupby
from pprint import pprint
from trueskill import Rating, rate

results = [('a', 'b', [1, 0]),
           ('a', 'c', [0, 1]),
           ('a', 'b', [0, 1]),
           ('b', 'c', [0, 1]),
           ('b', 'a', [1, 0]),
           ('c', 'b', [0, 1]),
           ('c', 'a', [1, 0]),
           ('c', 'a', [0, 0]),
           ('c', 'b', [0, 0]),
          ]

models = {
    'a': Rating(),
    'b': Rating(),
    'c': Rating(),
}

for game_id, (model_1, model_2, ranking) in enumerate(results):
    print('Pre-match', model_1, ':', models[model_1], model_2, ':', models[model_2], ranking)
    (models[model_1],), (models[model_2],) = rate([(models[model_1],), (models[model_2],)], ranks=ranking)
    print('Post-match', model_1, ':', models[model_1], model_2, ':', models[model_2])
    print()

pprint(models)

Pre-match a : trueskill.Rating(mu=25.000, sigma=8.333) b : trueskill.Rating(mu=25.000, sigma=8.333) [1, 0]
Post-match a : trueskill.Rating(mu=20.604, sigma=7.171) b : trueskill.Rating(mu=29.396, sigma=7.171)

Pre-match a : trueskill.Rating(mu=20.604, sigma=7.171) c : trueskill.Rating(mu=25.000, sigma=8.333) [0, 1]
Post-match a : trueskill.Rating(mu=25.046, sigma=6.265) c : trueskill.Rating(mu=19.003, sigma=6.873)

Pre-match a : trueskill.Rating(mu=25.046, sigma=6.265) b : trueskill.Rating(mu=29.396, sigma=7.171) [0, 1]
Post-match a : trueskill.Rating(mu=28.930, sigma=5.510) b : trueskill.Rating(mu=24.307, sigma=6.014)

Pre-match b : trueskill.Rating(mu=24.307, sigma=6.014) c : trueskill.Rating(mu=19.003, sigma=6.873) [0, 1]
Post-match b : trueskill.Rating(mu=26.141, sigma=5.499) c : trueskill.Rating(mu=16.607, sigma=6.093)

Pre-match b : trueskill.Rating(mu=26.141, sigma=5.499) a : trueskill.Rating(mu=28.930, sigma=5.510) [1, 0]
Post-match b : trueskill.Rating(mu=24.068, sigma=4.960) a

In [139]:
dir(models['a'])
print(models['a'].exposure)
print(models['b'].exposure)
print(models['c'].exposure)
# print(leaderboard)

16.93579844244241
10.235233050269352
12.66154839507252
